In [62]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [63]:

def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 

    x_n,y_n,z_n=cells.shape
    vertex_cnt=0
    has_added_vertex={}
    for x in range(x_n-1):
        for y in range(y_n-1):
            for z in range(z_n-1):
                case_nums=getCaseNum(x,y,z,thres,cells)
                face_vertexes=[]
                for case_num in case_nums:
                    if case_num==-1:
                        break
                    
                    corner1=(x+CaseNum2EdgeOffset[case_num][0],
                             y+CaseNum2EdgeOffset[case_num][1],
                             z+CaseNum2EdgeOffset[case_num][2])
                    corner2=(x+CaseNum2EdgeOffset[case_num][3],
                             y+CaseNum2EdgeOffset[case_num][4],
                             z+CaseNum2EdgeOffset[case_num][5])
                    
                    v1=cells[corner1]
                    v2=cells[corner2]
                    u=v1/(v1-v2)
                    interaction_point=(np.array(corner1)+u*(np.array(corner2)-np.array(corner1)))
                    if has_added_vertex.get(tuple(interaction_point))==None:
                        vertex_array[vertex_cnt]=interaction_point
                        has_added_vertex[tuple(interaction_point)]=vertex_cnt
                        vertex_cnt+=1
                    face_vertexes.append(has_added_vertex[tuple(interaction_point)])
                    if len(face_vertexes)==3:
                        face_array.append(face_vertexes)
                        face_vertexes=[]

    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.values())
    return np.array(vertex_array), np.array(face_array)

In [64]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
meshes={}
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    meshes[shape_name]=mesh
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
2.1141672134399414 s

Time taken by algorithm
----------------------------------------
2.0077970027923584 s


In [65]:
meshes["bob"].show()

In [66]:
meshes["spot"].show()